### 5.
O processamento de informação geográfica é uma ferramenta omnipresente no nosso dia a dia. Pretende-se
estudar treinos associados a vários desportos. Foi criado um dataset associados a vários utilizadores da aplicação
EndoMondo. Serão disponibilizado vários ficheiros json onde será possível explorar a dinâmica dos vários treinos.
Pretende-se que sejam explorados usando a biblioteca ipyleaflet. Pretende-se que seja desenhadas as etapas,
calculados o nº de km a subir e a descer, determinada a velocidade média (em todo o percurso e separadamente
nos troços a subir e a descer), a distância em plano, etc. Devem comparar a evolução do ritmo cardíaco em
vários percursos e relaciona-lo com as zonas de treino.

In [ ]:

import numpy as np
import ipyleaflet
import json
import matplotlib as plt

nome_ficheiro_json = 'endomondoHR_proper_subset.json' # <--- ATUALIZE COM O NOME REAL DO SEU FICHEIRO

dados_treinos = {}

with open(nome_ficheiro_json, 'r', encoding='utf-8') as f:
    dados_treinos = json.load(f)
    
ids_treinos = list(dados_treinos.keys())


# ISTO (PROVAVELMENTE) NÃO É O TRABALHO
if ids_treinos:
    numero_do_treino = 1
    id_treino = ids_treinos[numero_do_treino - 1]
    treino = dados_treinos[id_treino]

    print(f"\n--- Dados do primeiro treino (ID: {id_treino}) ---")
    print(f"Desporto: {treino.get('sport', 'Não especificado')}")
    print(f"ID de utilizador: {treino.get('userId', 'Não especificado')}")

    # Aceder às listas de dados. Usamos .get() para evitar erros caso a chave não exista.
    longitudes = treino.get('longitude', [])
    latitudes = treino.get('latitude', [])
    altitudes = treino.get('altitude', [])
    ritmo_cardiaco = treino.get('heart_rate', [])
    timestamps = treino.get('timestamp', [])

    print(f"Número de pontos de longitude: {len(longitudes)}")
    print(f"Número de pontos de latitude: {len(latitudes)}")
    print(f"Número de pontos de altitude: {len(altitudes)}")
    print(f"Número de pontos de ritmo cardíaco: {len(ritmo_cardiaco)}")
    print(f"Número de timestamps: {len(timestamps)}")

    num_pontos_exemplo = 5
    print(f"\nPrimeiros {num_pontos_exemplo} pontos de dados:")
    for i in range(min(num_pontos_exemplo, len(longitudes))):
        print(f"Ponto {i+1}: Lat={latitudes[i]}, Lon={longitudes[i]}, Alt={altitudes[i]}, FC={ritmo_cardiaco[i] if i < len(ritmo_cardiaco) else 'N/A'}, Timestamp={timestamps[i]}")

else:
    print("\nO dicionário de treinos está vazio.")
    
mapa = ipyleaflet.Map(center = (43, 0), zoom = 9)
line = ipyleaflet.Polyline(locations=dados_treinos[''], fill = False)
mapa.add_layer(line)
mapa


--- Dados do primeiro treino (ID: 588988811) ---
Desporto: bike (transport)
ID de utilizador: 4997910
Número de pontos de longitude: 500
Número de pontos de latitude: 500
Número de pontos de altitude: 500
Número de pontos de ritmo cardíaco: 500
Número de timestamps: 500

Primeiros 5 pontos de dados:
Ponto 1: Lat=-26.116309, Lon=28.098246, Alt=1637.4, FC=78, Timestamp=1440455857
Ponto 2: Lat=-26.116348, Lon=28.097663, Alt=1637.4, FC=75, Timestamp=1440455864
Ponto 3: Lat=-26.116369, Lon=28.097653, Alt=1637.4, FC=76, Timestamp=1440455871
Ponto 4: Lat=-26.116361, Lon=28.097629, Alt=1627.0, FC=80, Timestamp=1440455878
Ponto 5: Lat=-26.116485, Lon=28.097424, Alt=1624.0, FC=84, Timestamp=1440455885


KeyError: 'lating'